In [1]:
#read data
import sys #sys 模块包含了与 Python 解释器和它的环境有关的函数。
sys.path.append('..') #sys.path 包含输入模块的目录名列表
import utils

batch_size=256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

In [3]:
#initicalize parameters
from mxnet import ndarray as nd
num_inputs=28*28
num_outputs=10

num_hidden=256
weight_scale = .01

w1=nd.random_normal(shape=(num_inputs,num_hidden),scale=weight_scale)
b1=nd.zeros(num_hidden)

w2=nd.random_normal(shape=(num_hidden, num_outputs),scale=weight_scale)
b2=nd.zeros(num_outputs)

params = [w1, b1, w2, b2]

for param in params:
    param.attach_grad()



In [4]:
#define relu
def relu(X):
    return nd.maximum(X,0)


In [5]:
#define function 我们的模型就是将层（全连接）和激活函数（Relu）串起来：
def net(X):
    X = X.reshape((-1, num_inputs))
    h1=relu(nd.dot(X,w1)+b1)
    output=nd.dot(h1,w2)+b2
    return output


In [9]:
#define softmax function and cross entropy cost function  we use gluon 
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()


In [11]:
#training data
from mxnet import autograd
learning_rate = .5

for epoch in range(5):
    train_acc=0.
    train_loss=0.
    for data, label in train_data:
        with autograd.record():
            output=net(data)
            loss=softmax_cross_entropy(output,label)
        loss.backward()
        utils.SGD(params, learning_rate/batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
        
       
    test_acc = utils.evaluate_accuracy(test_data, net) 
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))
        
        


Epoch 0. Loss: 0.500754, Train acc 0.814002, Test acc 0.809796
Epoch 1. Loss: 0.431778, Train acc 0.839777, Test acc 0.853966
Epoch 2. Loss: 0.398572, Train acc 0.854284, Test acc 0.861679
Epoch 3. Loss: 0.373248, Train acc 0.863114, Test acc 0.866887
Epoch 4. Loss: 0.356938, Train acc 0.868506, Test acc 0.854768
